### 테이블 형태 데이터에서 누락된 값 식별

In [2]:
import pandas as pd
import numpy as np
from io import StringIO
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [3]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [4]:
df.isnull().mean()

A    0.000000
B    0.000000
C    0.333333
D    0.333333
dtype: float64

In [5]:
df.dropna(how='all')
df.dropna(thresh=4)
# Nan값이 아닌 값이 4개보다 작은 행을 삭제
df.dropna(subset=['C'])
# C열에 Nan값이 있는 행을 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


In [6]:
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


### 누락된 값 대체
#### 여러가지 보간 기법을 사용

In [8]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)

In [10]:
imputed_data = imr.transform(df.values)
imputed_data
#Nan값을 평균값으로 대체    
#strategy='median' 중앙값, 'most_frequent' 최빈값을 또 많이 사용한다
# 'most_frequent는 범주형 특성 데이터에 사용된다    

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [15]:
roz = df.fillna(method='bfill')
#누락된 값을 다음 행의 값으로 대체
roz = df.fillna(method='ffill')
#누락된 값을 이전 행의 값으로 대체
roz = df.fillna(method='ffill',axis=1)
#누락된 값을 이전 열의 값으로 대체
roz

C:\Users\82108\AppData\Local\Temp\ipykernel_23752\2209173024.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  roz = df.fillna(method='bfill')
C:\Users\82108\AppData\Local\Temp\ipykernel_23752\2209173024.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  roz = df.fillna(method='ffill')
C:\Users\82108\AppData\Local\Temp\ipykernel_23752\2209173024.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  roz = df.fillna(method='ffill',axis=1)


,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,6.0,8.0
2,10.0,11.0,12.0,12.0


### 사이킷런 추정기 API 익히기
SimpleImputer 클래시는 사이킷런의 변환기 API의 일부이다.
fit을 사용하여 데이터에서 모델 파라미터를 학습한다
transform을 사용하여 학습한 파라미터로 데이터를 변환한다
---
### 범주형 데이터 다루기
범주형 데이터는 순서가 있는 것과 없는 것으로 구별이 가능하다

In [16]:
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


### 순서가 있는 특성 매핑
학습 알고리즘이 순서 특성을 올바르게 인식하려면 범주형의 문자열 값을 정수로 바꿔야 한다
예를 들어 XL= L+1 = M+2 와 같은 것이다

In [17]:
size_mapping = {'XL': 3, 'L' : 2, 'M' : 1}
df['size'] = df['size'].map(size_mapping)

In [18]:
# 값을 되돌리고 싶다면 거꾸로 매핑하는 딕셔너리를 만들어서 map 메서드를 다시 호출
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

많은 머신러닝 라이브러리는 클래스 레이블이 정수로 인코딩 되었을 것이라고 기대한다
사이킷런의 분류 추정기 대부분은 클래스 레이블을 정수로 변환해 주지만 사소한 실수를 방지하기 위해
클래스 레이블을 정수 배열로 전달하는 것은 좋은 습관이다

In [19]:
import numpy as np
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [20]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [21]:
# 매핑 딕셔너리의 키-값 쌍을 뒺비어서 변환된 클래스 레이블을 다시 원본 문자열로 바꿀 수 있다
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [37]:
# 아니면 사이킷런에 구현된 LabelEncoder 클래스를 사용할 수 있다
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1], dtype=int64)

In [39]:
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [40]:
class_le.inverse_transform(y)


array([1, 0, 1], dtype=int64)

In [35]:
df['classlabel'] = class_le.inverse_transform(y)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [54]:
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


### 순서가 없는 특성에 원-핫 인코딩 적용

순서가 없는 열에도 비슷한 방식을 적용할 수 있다

In [42]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 'M', 10.1],
       [2, 'L', 13.5],
       [0, 'XL', 15.3]], dtype=object)

In [43]:
from sklearn.preprocessing import OneHotEncoder

X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [45]:
from sklearn.compose import ColumnTransformer
X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([ ('onehot', OneHotEncoder(),[0]), ('nothing', 'passthrough', [1, 2])])
c_transf.fit_transform(X)

array([[0.0, 1.0, 0.0, 'M', 10.1],
       [0.0, 0.0, 1.0, 'L', 13.5],
       [1.0, 0.0, 0.0, 'XL', 15.3]], dtype=object)

In [51]:
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [57]:
pd.get_dummies(df[['price','color','size']],columns=['size'])

,price,color_blue,color_green,color_red,size_L,size_M,size_XL
0,10.1,False,True,False,False,True,False
1,13.5,False,False,True,True,False,False
2,15.3,True,False,False,False,False,True


In [58]:

# get_dummies에서 다중 공선성 문제 처리
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

,price,color_green,color_red,size_M,size_XL
0,10.1,True,False,True,False
1,13.5,False,True,False,False
2,15.3,False,False,False,True


In [60]:
# OneHotEncoder에서 다중 공선성 문제 처리
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([ ('onehot', color_ohe, [0]),
                               ('nothing', 'passthrough', [1, 2])])
c_transf.fit_transform(X)

array([[1.0, 0.0, 'M', 10.1],
       [0.0, 1.0, 'L', 13.5],
       [0.0, 0.0, 'XL', 15.3]], dtype=object)

### 순서가 있는 특성 인코딩하기
순서가 있는 특성의 범주 사이에서 수치적 크기에 대해 확신이 없거나 두 범주 사이의 순서를 정의할 수 없다면 임계 값 을 사용하여 0/1로 인코딩할 수 있습니다.
예를 들어 M, L, XL 값을 가진 특성 size를 두 개의 새로운 특성 'x > M'과 'x > L'로 나눌 수 있습니다.

In [61]:
df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [62]:
df['x>M'] = df['size'].apply(lambda x: 1 if x in {'L', 'XL'} else 0)
df['x>L'] = df['size'].apply(lambda x: 1 if x == 'XL' else 0)
del df['size']
df

,color,price,classlabel,x>M,x>L
0,green,10.1,class2,0,0
1,red,13.5,class1,1,0
2,blue,15.3,class2,1,1


### 테스트 셋을 훈련 데이터셋과 테스트 데이터셋으로 나누기


In [63]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)

# UCI 머신러닝 저장소의 Wine 데이터셋에 접근되지 않을 때
# 다음 코드의 주석을 제거하고 로컬 경로에서 데이터셋을 읽으세요:

# df_wine = pd.read_csv('wine.data', header=None)


df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [65]:
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test =\
    train_test_split(X, y,
                     test_size=0.3,
                     random_state=0,
                     stratify=y)

대부분 정규화는 특성의 스케일을 [0,1] 범위에 맞추는 것을 의미한다
최소-최대 스케일 변환(min-max scaling)이 특별한 경우이다

In [66]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)
# 정해진 범위의 값이 필요할 때 유용하게 사용할 수 있는 일반적인 기법

표준화는 많은 머신 러닝 알고리즘, 특히 경사 하강법 같은 최적화 알고리즘에서 널리 사용된다
많은 머신 러닝 알고리즘, 특히 경사 하강법 같은 최적화 알고리즘에 주로 사용된다
표준화는 특성의 평균을 0에 맞추고 표준편차를 1로 맞추어 정규분포와 같은 특징을 갖도록 만든다
알고리즘이 이상치에 덜 민감하게 만든다

In [67]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

### 4.5 유용한 특성 선택
오차 감소를 위해
1. 더 많은 훈련 데이터를 모은다
2. 규제를 통해 복잡도를 제한한다
3. 파라미터 개수가 적은 간단한 모델을 선택한다
4. 데이터 차원을 줄인다